In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("C:/Users/毛嘉宇/jupyter/MNIST_data/", one_hot=True)
 
#每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size
 
#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)#标准差
        tf.summary.scalar('max', tf.reduce_max(var))#最大值
        tf.summary.scalar('min', tf.reduce_min(var))#最小值
        tf.summary.histogram('histogram', var)#直方图

#初始化权值
def weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.1)#生成一个截断的正态分布
    return tf.Variable(initial,name=name)
 
#初始化偏置
def bias_variable(shape,name):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial,name=name)
 
#卷积层
def conv2d(x,W):
    #x input tensor of shape `[batch, in_height, in_width, in_channels]`
    #W filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #`strides[0] = strides[3] = 1`. strides[1]代表x方向的步长，strides[2]代表y方向的步长
    #padding: A `string` from: `"SAME", "VALID"`
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
 
#池化层
def max_pool_2x2(x):
    #ksize [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
 
#命名空间
with tf.name_scope('input'):
    #定义两个placeholder
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')
    with tf.name_scope('x_image'):
        #改变x的格式转为 4D 的向量[batch, in_height, in_width, in_channels]`
        x_image = tf.reshape(x,[-1,28,28,1],name='x_image')
 
 
with tf.name_scope('Conv1'):
    #初始化第一个卷积层的权值和偏置
    with tf.name_scope('W_conv1'):
        W_conv1 = weight_variable([5,5,1,32],name='W_conv1')#5*5的采样窗口，32个卷积核从1个平面抽取特征
    with tf.name_scope('b_conv1'):  
        b_conv1 = bias_variable([32],name='b_conv1')#每一个卷积核一个偏置值
 
    #把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
    with tf.name_scope('conv2d_1'):
        conv2d_1 = conv2d(x_image,W_conv1) + b_conv1
    with tf.name_scope('relu'):
        h_conv1 = tf.nn.relu(conv2d_1)
    with tf.name_scope('h_pool1'):
        h_pool1 = max_pool_2x2(h_conv1)#进行max-pooling

        
with tf.name_scope('Conv2'):
    #初始化第二个卷积层的权值和偏置
    with tf.name_scope('W_conv2'):
        W_conv2 = weight_variable([5,5,32,64],name='W_conv2')#5*5的采样窗口，64个卷积核从32个平面抽取特征
    with tf.name_scope('b_conv2'):  
        b_conv2 = bias_variable([64],name='b_conv2')#每一个卷积核一个偏置值
 
    #把h_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
    with tf.name_scope('conv2d_2'):
        conv2d_2 = conv2d(h_pool1,W_conv2) + b_conv2
    with tf.name_scope('relu'):
        h_conv2 = tf.nn.relu(conv2d_2)
    with tf.name_scope('h_pool2'):
        h_pool2 = max_pool_2x2(h_conv2)#进行max-pooling

#28*28的图片第一次卷积后还是28*28，第一次池化后变为14*14
#第二次卷积后为14*14，第二次池化后变为了7*7
#进过上面操作后得到64张7*7的平面
 
with tf.name_scope('fc1'):
    #初始化第一个全连接层的权值
    with tf.name_scope('W_fc1'):
        W_fc1 = weight_variable([7*7*64,1024],name='W_fc1')#上一场有7*7*64个神经元，全连接层有1024个神经元
    with tf.name_scope('b_fc1'):
        b_fc1 = bias_variable([1024],name='b_fc1')#1024个节点
 
    #把池化层2的输出扁平化为1维
    with tf.name_scope('h_pool2_flat'):
        h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64],name='h_pool2_flat')
    #求第一个全连接层的输出
    with tf.name_scope('wx_plus_b1'):
        wx_plus_b1 = tf.matmul(h_pool2_flat,W_fc1) + b_fc1
    with tf.name_scope('relu'):
        h_fc1 = tf.nn.relu(wx_plus_b1)
 
    #keep_prob用来表示神经元的输出概率
    with tf.name_scope('keep_prob'):
        keep_prob = tf.placeholder(tf.float32,name='keep_prob')
    with tf.name_scope('h_fc1_drop'):
        h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob,name='h_fc1_drop')

        
with tf.name_scope('fc2'):
    #初始化第二个全连接层
    with tf.name_scope('W_fc2'):
        W_fc2 = weight_variable([1024,10],name='W_fc2')
    with tf.name_scope('b_fc2'):    
        b_fc2 = bias_variable([10],name='b_fc2')
    with tf.name_scope('wx_plus_b2'):
        wx_plus_b2 = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
    with tf.name_scope('softmax'):
        #计算输出
        prediction = tf.nn.softmax(wx_plus_b2)

        
#交叉熵代价函数
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction),name='loss')
    tf.summary.scalar('loss',cross_entropy)
     
#使用AdamOptimizer进行优化
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(2e-4).minimize(cross_entropy)

    
#求准确率
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #结果存放在一个布尔列表中
        correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))#argmax返回一维张量中最大的值所在的位置
    with tf.name_scope('accuracy'):
        #求准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
         
#合并所有的summary
merged = tf.summary.merge_all()
 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('6_1_graph/train',sess.graph)
    test_writer = tf.summary.FileWriter('6_1_graph/test',sess.graph)
    for i in range(4001):
        #训练模型
        batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
        sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.25})
        #记录训练集计算的参数
        summary = sess.run(merged,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.25})
        train_writer.add_summary(summary,i)
        #记录测试集计算的参数
        batch_xs,batch_ys =  mnist.test.next_batch(batch_size)
        summary = sess.run(merged,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.25})
        test_writer.add_summary(summary,i)
     
        if i%100==0:
            test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
            train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images[:3000],y:mnist.train.labels[:3000],keep_prob:1.0})
            print ("Iter  " + str(i) + " ,  \t Testing Accuracy= " + str(test_acc) + ",  \t Training Accuracy= " + str(train_acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting C:/Users/毛嘉宇/jupyter/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting C:/Users/毛嘉宇/jupyter/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting C:/Users/毛嘉宇/jupyter/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting C:/Users/毛嘉宇/jupyter/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Iter  0 ,  	 Testing Accur

In [3]:
'''
1.   卷积操作
     卷积是 3*3 的矩阵且步长为 1.
[ 1  1  0  1  0
  0  0  1  0  1             [1  0  1          [2  5  2              [ 1  0  1  
  0  1  1  1  1       *      0  1  0    ==     4  1  5         除了   0  1  0   这一卷积层外,还可以使用其他卷积提取图片特征    
  1  0  1  0  1              1  0  1]          1  5  2].              1  0  1 ]
  0  1  0  1  0 ]
  
  例如：
  
   [0   0   0                            [-1  -1  -1                     [-1  -1   0
    1   5   0   凸显下方与中间特征，或者  -1   8  -1  凸显中间特征，或者  -1   0   1  凸显右下于左上的特征区别等等.
    1   1   1]                            -1  -1  -1]                      0   1   1]


2.   池化操作
[ 1 1   1 0             
  2 3   3 1                  [3    3                              [7/4   5/4
                 max-pooling                          mean-pooling     
  2 3   2 1                   3    2],                              2    3/2]
  1 2   2 1]
  
3.   padding 

(1)  same padding:  给‘原图像’的‘像素外圈’补 0, 使得卷积采样后能采集所有的点(注意步长).
                    步长为 1 时，得到一个和‘原图像像素’大小相同的‘新的图像’.

(2) valid padding:  采样不会超过‘原图像’的‘像素外圈’.

4.   池化操作中的 padding

(1)  same padding:  原图 28 * 28， 卷积 2 * 2，步长为 2. 此时不用补 0 可采取全部‘像素’，样本为 14 * 14.
                    原图  2 * 3 ， 卷积 2 * 2，步长为 2. 此时在第 4 列补 0 可采取全部‘像素’，样本为 1 * 2.
(2) valid padding:  原图 28 * 28， 卷积 2 * 2，步长为 2. 此时采取全部像素后，样本为 14 * 14.
                    原图  2 * 3 ， 卷积 2 * 2，步长为 2. 此时无法采取全部‘像素’，放弃第 3 列，样本为 1 * 1.

'''

'\n1.   卷积操作\n     卷积是 3*3 的矩阵且步长为 1.\n[ 1  1  0  1  0\n  0  0  1  0  1             [1  0  1          [2  5  2              [ 1  0  1  \n  0  1  1  1  1       *      0  1  0    ==     4  1  5         除了   0  1  0   这一卷积层外,还可以使用其他卷积提取图片特征    \n  1  0  1  0  1              1  0  1]          1  5  2].              1  0  1 ]\n  0  1  0  1  0 ]\n  \n  例如：\n  \n   [0   0   0                            [-1  -1  -1                     [-1  -1   0\n    1   5   0   凸显下方与中间特征，或者  -1   8  -1  凸显中间特征，或者  -1   0   1  凸显右下于左上的特征区别等等.\n    1   1   1]                            -1  -1  -1]                      0   1   1]\n\n\n2.   池化操作\n[ 1 1   1 0             \n  2 3   3 1                  [3    3                              [7/4   5/4\n                 max-pooling                          mean-pooling     \n  2 3   2 1                   3    2],                              2    3/2]\n  1 2   2 1]\n  \n3.   padding \n\n(1)  same padding:  给‘原图像’的‘像素外圈’补 0, 使得卷积采样后能采集所有的点(注意步长).\n                    